In [32]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
import seaborn as sns
import functions as f
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor

In [33]:
# better visualization of numbers
pd.options.display.float_format = '{:,.2f}'.format 

# Diary

I tried different methods to fill missing values such as linear regression and knnImputer. Because the initial idea was comparing different model accuracy then select the best one. But as i have a lot of categorical data which is difficult and no sense to transform into numerical values I discarded these ideas. Then i filled missing values using random forest reressor. 

# Functions


## Linear regression

I wanted to use linear regression to fill data then compare its accuracy with random forest accuracy, but having a lot of categorical data I wasn't able to do, so I discard this idea. I will impute missing values directly with random forest

In [34]:
# def prepare_data(df, target, columns_to_exclude):
#     features = [col for col in df.columns if col not in columns_to_exclude + [target]]
#     # One-Hot Encoding for categorical features
#     df_encoded = pd.get_dummies(df[features], drop_first=True)
#     X = df[features]
#     y = df[target]

#     return X, y

# def evaluate_and_plot(X_test, y_test, model, target):
#     # make prediction
#     y_pred = model.predict(X_test)

#     # model evaluation
#     MSE = mean_squared_error(y_test, y_pred)
#     print(f'Mean Squared Error: {MSE}')
#     MAE = mean_absolute_error(y_test, y_pred)
#     print(f'MAE score is: {MAE}')
#     RMSE = np.sqrt(MSE)
#     print(f'RMSE score is: {RMSE}')
#     R2_Score = r2_score(y_test, y_pred)
#     print(f'R2_Score score is: {R2_Score}')


# def fill_missin_value(df, model, columns_to_exclude=[]):
    
#     for target in df.columns:
#         if df[target].isna().any:
#             target_with_values = df[df[target].notna()]     # complete data to use to rain the model
#             target_without_values =df[df[target].isna()]    # used to predict and fill the missing values

#             if not target_without_values.empty:
#                 X_complete, y_complete = prepare_data(target_with_values, target, columns_to_exclude=[])      
#                 # X_complete = subset of the data with no missing values in target column
#                 X_incomplete, _ = prepare_data(target_without_values, target, columns_to_exclude=[])
#                 # we use '_' to ignore second value returned by prepare_data function bc we use only X_incomplete to make predictions

#                 model.fit(X_complete, y_complete)
#                 y_pred = model.predict(X_incomplete)

#                 df.loc[df[target].isna(), target] = y_pred

#     return df


## Random Forest

In [35]:
def impute_missing_values_with_random_forest(df, column_name):
    # Select only numerical columns (excluding the target column)
    numerical_columns = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
    if column_name in numerical_columns:
        numerical_columns.remove(column_name)

    # Exclude columns with NaN values
    numerical_columns = [col for col in numerical_columns if not df[col].isna().any()]

    # Splitting the data into two parts: one where target column is missing and one where it's not
    df_with_target = df[df[column_name].notna()] #the target column is not NaN: These rows provide 
                        # valuable information because they contain actual data that the model can learn from
    df_without_target = df[df[column_name].isna()] #rows where the target column is NaN and needs to be imputed 
                                # (using the information from the rows where the target column is not empty.)

    # Prepare the features (X) and target (y) using only numerical columns
    x = df_with_target[numerical_columns]
    y = df_with_target[column_name]

    # Create and train the model
    model = RandomForestRegressor(random_state=42)
    model.fit(x, y)

    # Predicting the missing values
    predicted_values = model.predict(df_without_target[numerical_columns])

    # Fill in the missing values in the original DataFrame
    df.loc[df[column_name].isna(), column_name] = predicted_values
    return df

### Plotting model results

In [36]:
def plot_model(y_test, y_pred, target):
    plt.figure(figsize=(10, 6))
    plt.plot(y_test.values[:100], label='Actual')
    plt.plot(y_pred[:100], label='Predicted')
    plt.xlabel('Sample')
    plt.ylabel(target)
    plt.title(f'Actual vs Predicted {target}')
    plt.legend()
    plt.show()

# Dataset Informations

**COLUMNS**
1. **ID:** identifier for each athlete.
2. **NAME:**  The full name of the athlete.
3. **SEX:** The gender of the athlete, represented as 'M' for male and 'F' for female.
4. **AGE:** The age of the athlete at the time of the Olympics.
5. **HEIGHT:** The height of the athlete in centimeters.
6. **WEIGHT:** The weight of the athlete in kilograms.
7. **TEAM:** The country the athlete represents.
8. **NOC:** The National Olympic Committee (NOC) code for the country the athlete represents.
9. **GAMES:** The edition of the Olympics the athlete participated in, including the year and the season (Summer or Winter).
10. **YEAR:** The year of the Olympics.
11. **SEASON:** The season of the Olympics, either Summer or Winter.
12. **CITY:** The host city of the Olympics.
13. **SPORT:** The sport the athlete competed in.
14. **EVENT:** The specific event within the sport that the athlete competed in.
15. **MEDAL:** The type of medal won by the athlete, if any (Gold, Silver, Bronze, or NaN if no medal was won).


## IDEAS

TO DO for dataset exploration: 
1. compare male and female athlets participation to the games each year
2. correlation between age, sex, height and weight.
3. distribution of medals per age and per sex
4. Check if medals are distribuited correctly, in sense of: if there are years that have more medal than the maximum they can have. For this study the olympics description on kaggle. 
5. LOOK IF IT MAKES SENSE TO ASSOCIATE AN ID TO EACH COUNTRY IN *noc_region.csv* DATASET AND USE IT FOR ANALYSING THE SUCCESS OF THEM?
6. Should I transform float data in int? not all of them, like medals etc ?

# DATA EXPLORATION AND DATA WRANGLING

In [37]:
olympic_df = pd.read_csv('dataset_olympic_data/dataset_olympics.csv')
noc_region_df = pd.read_csv('dataset_olympic_data/noc_region.csv')

## EXPLORING THE DATASET

In [38]:
noc_region_df

,noc_region,reg,notes
0,AFG,Afghanistan,NaN
1,AHO,Curacao,Netherlands Antilles
2,ALB,Albania,NaN
3,ALG,Algeria,NaN
4,AND,Andorra,NaN
...,...,...,...
225,YEM,Yemen,NaN
226,YMD,Yemen,South Yemen
227,YUG,Serbia,Yugoslavia
228,ZAM,Zambia,NaN


In [39]:
noc_region_df['notes'].unique()

array([nan, 'Netherlands Antilles', 'Antigua and Barbuda', 'Australasia',
       'Bohemia', 'Crete', 'Hong Kong', 'Individual Olympic Athletes',
       'Virgin Islands', 'North Borneo', 'Newfoundland',
       'Refugee Olympic Team', 'Serbia and Montenegro',
       'Turks and Caicos Islands', 'Trinidad and Tobago', 'Tuvalu',
       'United Arab Republic', 'Unknown', 'West Indies Federation',
       'North Yemen', 'South Yemen', 'Yugoslavia'], dtype=object)

In [40]:
noc_region_df[noc_region_df['notes'] == 'Newfoundland']

,noc_region,reg,notes
147,NFL,Canada,Newfoundland


In [41]:
olympic_df.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.00,180.00,80.00,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.00,170.00,60.00,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.00,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,4,Edgar Lindenau Aabye,M,34.00,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.00,185.00,82.00,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN


In [42]:
olympic_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70000 entries, 0 to 69999
Data columns (total 15 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ID      70000 non-null  int64  
 1   Name    70000 non-null  object 
 2   Sex     70000 non-null  object 
 3   Age     67268 non-null  float64
 4   Height  53746 non-null  float64
 5   Weight  52899 non-null  float64
 6   Team    70000 non-null  object 
 7   NOC     70000 non-null  object 
 8   Games   70000 non-null  object 
 9   Year    70000 non-null  int64  
 10  Season  70000 non-null  object 
 11  City    70000 non-null  object 
 12  Sport   70000 non-null  object 
 13  Event   70000 non-null  object 
 14  Medal   9690 non-null   object 
dtypes: float64(3), int64(2), object(10)
memory usage: 8.0+ MB


In [43]:
olympic_df.describe()

,ID,Age,Height,Weight,Year
count,"70,000.00","67,268.00","53,746.00","52,899.00","70,000.00"
mean,"18,081.85",25.64,175.51,70.90,"1,977.77"
std,"10,235.61",6.49,10.38,14.22,30.10
min,1.00,11.00,127.00,25.00,"1,896.00"
25%,"9,325.75",21.00,168.00,61.00,"1,960.00"
50%,"18,032.00",25.00,175.00,70.00,"1,984.00"
75%,"26,978.00",28.00,183.00,79.00,"2,002.00"
max,"35,658.00",88.00,223.00,214.00,"2,016.00"


In [44]:
olympic_df.isna().sum()

ID            0
Name          0
Sex           0
Age        2732
Height    16254
Weight    17101
Team          0
NOC           0
Games         0
Year          0
Season        0
City          0
Sport         0
Event         0
Medal     60310
dtype: int64

In [45]:
olympic_df['Medal'].unique()

array([nan, 'Gold', 'Bronze', 'Silver'], dtype=object)

In [46]:
medal_mask = ['Gold', 'Silver', 'Bronze']
olympic_df['Medal'].value_counts()[medal_mask] 

Medal
Gold      3292
Silver    3190
Bronze    3208
Name: count, dtype: int64

In [47]:
# olympic_df.hist()

In [48]:
# olympic_without_id = olympic_df.drop(columns='ID')
# olympic_without_id
# # plt.figure(figsize=(10, 6))
# # olympic_corr = sns.heatmap(olympic_without_id.corr(), annot=True)
# # olympic_corr.set_title('Olympic Data Correlation Matrix')

## Data Cleaning

COLUMNS CAN BE TRANSFORM:
- SEX: M/F -> 0/1
- MEDAL: NaN, Bronze, Silver, Gold -> 0, 1, 2, 3

I can use two methods to encode the labels:
1. LabelEncoder from sklearn library
2. Doing it by hand

As I have just few labels to encode, I prefer to use second method because LabelEncoder assign numbers starting from 0 and so on and in my case is not preferable a generic assignment because of the different priorty/importance of the medal types. Instead for Sex encoding it can be used LabelEncoding but for code clearity I will use one method for all my features that I want to encode.

In [49]:
# col = olympic_df.columns
olympic_df['Medal'].unique()
# f.print_unique_values(olympic_df[col])

array([nan, 'Gold', 'Bronze', 'Silver'], dtype=object)

In [50]:
# label_encoder = preprocessing.LabelEncoder() 
# olympic_df['Sex']= label_encoder.fit_transform(olympic_df['Sex']) 

In [51]:
replace_sex={
    'M':0,
    "F":1
}
olympic_df['Sex'] = olympic_df['Sex'].replace(replace_sex)

replace_medal={
    "Bronze":1,
    "Silver":2,
    "Gold":3
}
olympic_df['Medal'] = olympic_df['Medal'].replace(replace_medal)

pd.set_option("future.no_silent_downcasting", True)

### Check duplicated

In [52]:
olympic_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70000 entries, 0 to 69999
Data columns (total 15 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ID      70000 non-null  int64  
 1   Name    70000 non-null  object 
 2   Sex     70000 non-null  object 
 3   Age     67268 non-null  float64
 4   Height  53746 non-null  float64
 5   Weight  52899 non-null  float64
 6   Team    70000 non-null  object 
 7   NOC     70000 non-null  object 
 8   Games   70000 non-null  object 
 9   Year    70000 non-null  int64  
 10  Season  70000 non-null  object 
 11  City    70000 non-null  object 
 12  Sport   70000 non-null  object 
 13  Event   70000 non-null  object 
 14  Medal   9690 non-null   object 
dtypes: float64(3), int64(2), object(10)
memory usage: 8.0+ MB


In [53]:
print('There are {} duplicate rows in olympic_df based on all columns.'.format(olympic_df.duplicated().sum()))


There are 383 duplicate rows in olympic_df based on all columns.


In [54]:
olympic_df.drop_duplicates(keep='first', inplace=True)

In [55]:
print('There are {} duplicate rows in olympic_df based on all columns.'.format(olympic_df.duplicated().sum()))


There are 0 duplicate rows in olympic_df based on all columns.


In [56]:
olympic_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 69617 entries, 0 to 69999
Data columns (total 15 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ID      69617 non-null  int64  
 1   Name    69617 non-null  object 
 2   Sex     69617 non-null  object 
 3   Age     66946 non-null  float64
 4   Height  53741 non-null  float64
 5   Weight  52899 non-null  float64
 6   Team    69617 non-null  object 
 7   NOC     69617 non-null  object 
 8   Games   69617 non-null  object 
 9   Year    69617 non-null  int64  
 10  Season  69617 non-null  object 
 11  City    69617 non-null  object 
 12  Sport   69617 non-null  object 
 13  Event   69617 non-null  object 
 14  Medal   9686 non-null   object 
dtypes: float64(3), int64(2), object(10)
memory usage: 8.5+ MB


### Missing values

#### Percentage of missing data

In [24]:
missing_percentage = ((olympic_df.isnull().sum())/len(olympic_df))*100
missing_data = pd.DataFrame({'Missing Values': olympic_df.isnull().sum(), 'Percentage': missing_percentage})
print(missing_data)

        Missing Values  Percentage
ID                   0        0.00
Name                 0        0.00
Sex                  0        0.00
Age               2671        3.84
Height           15876       22.80
Weight           16718       24.01
Team                 0        0.00
NOC                  0        0.00
Games                0        0.00
Year                 0        0.00
Season               0        0.00
City                 0        0.00
Sport                0        0.00
Event                0        0.00
Medal            59931       86.09


#### Medal column cleaning

Filling missing values in Medal column with fillna(), because actually they are not missing values where we have NaN, but it indicates that the athlet didn't get any medal

In [25]:
olympic_df['Medal'] = olympic_df['Medal'].fillna(value=0)

In [26]:
olympic_df.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,0,24.00,180.00,80.00,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,0.00
1,2,A Lamusi,0,23.00,170.00,60.00,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,0.00
2,3,Gunnar Nielsen Aaby,0,24.00,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,0.00
3,4,Edgar Lindenau Aabye,0,34.00,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,3.00
4,5,Christine Jacoba Aaftink,1,21.00,185.00,82.00,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,0.00


In [27]:
olympic_df.isna().sum()

ID            0
Name          0
Sex           0
Age        2671
Height    15876
Weight    16718
Team          0
NOC           0
Games         0
Year          0
Season        0
City          0
Sport         0
Event         0
Medal         0
dtype: int64

In [28]:
olympic_df["Medal"].unique()

array([0., 3., 1., 2.])

#### Linear Regression to impute values : not used

Impute missing data with linear regression

In [29]:
# columns_to_exclude = olympic_df[['ID']]
# model = LinearRegression()

# olympic_df_filled = fill_missin_value(olympic_df, model, columns_to_exclude)  
         

#### Random Forest

In [30]:
for column in olympic_df.columns:
    if olympic_df[column].isna().any():
        olympic_df = impute_missing_values_with_random_forest(olympic_df, column)

In [31]:
olympic_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 69617 entries, 0 to 69999
Data columns (total 15 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ID      69617 non-null  int64  
 1   Name    69617 non-null  object 
 2   Sex     69617 non-null  int64  
 3   Age     69617 non-null  float64
 4   Height  69617 non-null  float64
 5   Weight  69617 non-null  float64
 6   Team    69617 non-null  object 
 7   NOC     69617 non-null  object 
 8   Games   69617 non-null  object 
 9   Year    69617 non-null  int64  
 10  Season  69617 non-null  object 
 11  City    69617 non-null  object 
 12  Sport   69617 non-null  object 
 13  Event   69617 non-null  object 
 14  Medal   69617 non-null  float64
dtypes: float64(4), int64(3), object(8)
memory usage: 8.5+ MB


## Plots

# MODEL 

# CONCLUSION